# Cirq/Openfermion notes

In [3]:
import math
import os
from typing import Callable, Iterable, List, Set

import matplotlib.pyplot as plt
import numpy as np
import pprint
from tqdm.notebook import tqdm

import cirq
import openfermion as of

A cirq pauli string can be created on a line of line qubits

In [4]:
a, b, c, d = cirq.LineQubit.range(4) #4 qubits
pauli1 = cirq.Z.on(a) * cirq.Z.on(b) * cirq.X.on(c) * cirq.X.on(d) #places paulis on slots

print(
    cirq.LineQubit.range(4), '\n',
    cirq.Z.on(a), '\n',
    pauli1, '\n',
    type(pauli1) #an operator, quantum circuits are a bunch of operations on an initial state
    )

[cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3)] 
 Z(q(0)) 
 Z(q(0))*Z(q(1))*X(q(2))*X(q(3)) 
 <class 'cirq.ops.pauli_string.PauliString'>


Openfermion also has paulis, like the hamiltonians in hamlib lets import one

In [10]:
import h5py

def read_openfermion_hdf5(fname_hdf5: str, key: str, optype=of.QubitOperator):
    """
    Read any openfermion operator object from HDF5 file at specified key.
    'optype' is the op class, can be of.QubitOperator or of.FermionOperator.
    """

    with h5py.File(fname_hdf5, 'r', libver='latest') as f:
        op = optype(f[key][()].decode("utf-8"))
    return op


def parse_through_hdf5(func):
    """
    Decorator function that iterates through an HDF5 file and performs
    the action specified by ‘ func ‘ on the internal and leaf nodes in the HDF5 file.
    """

    def wrapper (obj, path = '/', key = None) :
        if type(obj) in [h5py._hl.group.Group, h5py._hl.files.File]:
            for ky in obj.keys() :
                func(obj, path, key=ky, leaf = False)
                wrapper(obj = obj[ky], path = path + ky + ',', key = ky)
        elif type (obj) == h5py._hl.dataset.Dataset:
            func(obj, path, key = None, leaf = True)
    return wrapper


def get_hdf5_keys ( fname_hdf5 : str ) :
    """ Get a list of keys to all datasets stored in the HDF5 file .
    Args
    ----
    fname_hdf5 ( str ) : full path where HDF5 file is stored
    """

    all_keys = []
    @parse_through_hdf5
    def action(obj, path = '/', key = None, leaf = False):
        if leaf is True :
            all_keys.append(path)

    with h5py.File(fname_hdf5, 'r') as f:
        action(f['/'])
    return all_keys

In [31]:
molecule_to_read = 'O2'

hamiltonian = read_openfermion_hdf5(
    './'+molecule_to_read+'.hdf5',
    get_hdf5_keys('./'+molecule_to_read+'.hdf5')[0].rstrip(",")
)

Again, an operator. but now a bit more complicated

In [49]:
def printer(*args):
    for arg in args:
        print(arg)

printer(
    type(hamiltonian),
    hamiltonian.terms.keys(),
    list(hamiltonian.terms.items())[3]
)

#for term in hamiltonian.terms:
#    for x,y in term:
#        print(x,y)

<class 'openfermion.ops.operators.qubit_operator.QubitOperator'>
dict_keys([(), ((0, 'X'), (1, 'X'), (2, 'X'), (3, 'Y'), (5, 'Y')), ((0, 'X'), (1, 'X'), (2, 'Z'), (3, 'Y'), (4, 'Y'), (5, 'X')), ((0, 'X'), (1, 'X'), (3, 'X'), (4, 'Y'), (5, 'Y'), (6, 'Z'), (7, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (4, 'Z'), (5, 'Y'), (6, 'Y'), (7, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'X'), (8, 'Y'), (9, 'X'), (11, 'Y'), (12, 'Z'), (13, 'X'), (15, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'X'), (8, 'Y'), (9, 'Y'), (10, 'Z'), (15, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'X'), (8, 'Z'), (9, 'X'), (11, 'Y'), (12, 'Y'), (13, 'X'), (15, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'X'), (8, 'Z'), (9, 'Y'), (10, 'Y'), (15, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'Y'), (8, 'X'), (9, 'X'), (11, 'X'), (12, 'Y'), (13, 'Y'), (14, 'Y')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'Y'), (8, 'X'), (9, 'Y'), (10, 'Y'), (11, 'Z'), (13, 'Z'), (14, 'Y')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'Y'), (8, 'Y'), (9, 'X'), (11, 'X')

You can create your own of operator using lists!

In [66]:
onebit = of.QubitOperator((0,'Z'),0.5)
twobit = of.QubitOperator(((0,'Z'),(1,'X')),1.1)
bitbybit = onebit,twobit #not quite the same format as the hamlib hamiltonians
printer(
    onebit,
    type(twobit),
    bitbybit
)

0.5 [Z0]
<class 'openfermion.ops.operators.qubit_operator.QubitOperator'>
(0.5 [Z0], 1.1 [Z0 X1])
